In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import torch
import os
from timeit import default_timer as timer

from sklearn import manifold
from sklearn.manifold.t_sne import trustworthiness
from sklearn.model_selection import KFold
from sklearn import neighbors
from sklearn.metrics import accuracy_score

In [2]:
from core.SimpleConvNet import SimpleConvNet
from core.GraphConvNet2 import GraphConvNet2
from core.DataEmbeddingGraph import DataEmbeddingGraph
from util.plot_embedding import plot_embedding, plot_embedding_subplot
from util.mnist_data_loader import get_train_set, get_test_set


In [3]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'gpu'
else:
    print('cuda not available')
    device = 'cpu'

cuda not available


In [10]:
task_parameters = {}
task_parameters['reduction_method'] = 'spectral'
task_parameters['n_components'] = 2

net_parameters = {}
net_parameters['n_components'] = task_parameters['n_components']
net_parameters['D'] = 784 # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 10 # number of hidden layers
net_parameters['n_channels'] = 1
net_parameters['n_units_1'] = net_parameters['n_units_2'] = net_parameters['H']

In [5]:
filename = 'data/set_100_mnist_spectral_size_200_500.pkl'
with open(filename, 'rb') as f:
    [all_test_data] = pickle.load(f)

In [6]:
# Loss functions
def l2_norm(v):
    return np.sqrt(np.sum(np.square(v), axis=1, dtype=np.float64))

def pairwise_loss_function_1(y_true, y_pred, W):
    pairwise_loss_1 = mean_squared_error(y_true[W.row,:], y_true[W.col,:])
    pairwise_loss_2 = mean_squared_error(y_pred[W.row,:], y_pred[W.col,:])
    return np.square(pairwise_loss_1 - pairwise_loss_2)

def pairwise_loss_function_2(y_true, y_pred, W):
    pairwise_loss_1 = l2_norm(y_true[W.row,:] - y_true[W.col,:])
    pairwise_loss_2 = l2_norm(y_pred[W.row,:] - y_pred[W.col,:])
    return np.average(np.square(pairwise_loss_1 - pairwise_loss_2))

In [7]:
def one_nearest_neighbours_generalisation_error(X, y):
    kf = KFold(n_splits=10)
    kf.get_n_splits(X)
    errors = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = neighbors.KNeighborsClassifier(n_neighbors=1)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        errors.append(1- accuracy_score(y_test, y_pred))
    return np.average(errors)

In [8]:
def evaluate_net(net, all_test_data):
    n_test = len(all_test_data)
    similarity_tracker = np.zeros((n_test, ))
    trust_tracker = np.zeros((n_test, ))
    one_nn_tracker = np.zeros((n_test, ))
    time_tracker = np.zeros((n_test, ))
    
    for i in range(n_test):
        G = all_test_data[i]
        time_start = timer()
        y_pred = net.forward(G).detach().numpy()
        time_end = timer()
        time_tracker[i] = time_end - time_start
        
        similarity_tracker[i] =  pairwise_loss_function_2(G.target, y_pred, G.adj_matrix)
        X = G.data.view(G.data.shape[0], -1).numpy()
        trust_tracker[i] = trustworthiness(X, y_pred, n_neighbors=5)
        one_nn_tracker[i] = one_nearest_neighbours_generalisation_error(y_pred, G.labels.numpy())
    return similarity_tracker, trust_tracker, one_nn_tracker, time_tracker

In [111]:
def evaluate_original_embedding(all_test_data):
    n_test = len(all_test_data)
    trust_tracker = np.zeros((n_test, ))
    one_nn_tracker = np.zeros((n_test, ))
    time_tracker = np.zeros((n_test, )) 
    
    for i in range(n_test):
        G = all_test_data[i]
        X = G.data.view(G.data.shape[0], -1).numpy()
        time_start = timer()
        embedder = manifold.SpectralEmbedding(n_components=2, random_state=0,
                                              eigen_solver="arpack")
        X_spectral = embedder.fit_transform(X)
        time_end = timer()
        time_tracker[i] = time_end - time_start
        
        trust_tracker[i] = trustworthiness(X, X_spectral, n_neighbors=5)
        one_nn_tracker[i] = one_nearest_neighbours_generalisation_error(X_spectral, G.labels.numpy())
    return trust_tracker, one_nn_tracker, time_tracker

In [112]:
def evaluate_tsne_embedding(all_test_data):
    n_test = len(all_test_data)
    trust_tracker = np.zeros((n_test, ))
    one_nn_tracker = np.zeros((n_test, ))
    time_tracker = np.zeros((n_test, ))
    
    for i in range(n_test):
        G = all_test_data[i]
        X = G.data.view(G.data.shape[0], -1).numpy()
        time_start = timer()
        embedder = manifold.TSNE(n_components=2, init='pca', random_state=0)
        X_tsne = embedder.fit_transform(X)
        time_end = timer()
        time_tracker[i] = time_end - time_start
        
        trust_tracker[i] = trustworthiness(X, X_tsne, n_neighbors=5)
        one_nn_tracker[i] = one_nearest_neighbours_generalisation_error(X_tsne, G.labels.numpy())
    return trust_tracker, one_nn_tracker, time_tracker

In [113]:
# Spectral embeddings
trust_tracker, one_nn_tracker, time_tracker = evaluate_original_embedding(all_test_data)
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Trustworthy score = 0.7582520548568904
1-NN generalisation error = 0.5524332002099447
Average time to compute (s) = 0.17640546113987512


In [116]:
# t-SNE embeddings
trust_tracker, one_nn_tracker, time_tracker = evaluate_tsne_embedding(all_test_data)
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Trustworthy score = 0.955044333450673
1-NN generalisation error = 0.18022943449431011
Average time to compute (s) = 5.86404038087021


In [114]:
# Simple conv net
net = SimpleConvNet(net_parameters)
root = 'results/mnist_spectral1/'
filename = root + 'conv_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

similarity_tracker, trust_tracker, one_nn_tracker, time_tracker = evaluate_net(net, all_test_data)
print("Similarity loss = {}".format(np.average(similarity_tracker)))
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Similarity loss = 0.0709895799395694
Trustworthy score = 0.5501872305425336
1-NN generalisation error = 0.8476193517441856
Average time to compute (s) = 0.7939383930801704


In [11]:
# Graph net
net = GraphConvNet2(net_parameters)
root = 'results/mnist_tsne3/'
filename = root + 'graph_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

similarity_tracker, trust_tracker, one_nn_tracker, time_tracker = evaluate_net(net, all_test_data)
print("Similarity loss = {}".format(np.average(similarity_tracker)))
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Similarity loss = 214.97720496898555
Trustworthy score = 0.771702325260192
1-NN generalisation error = 0.5683302793371392
Average time to compute (s) = 0.370508154551062


In [10]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np

from core.GraphConvNetCell import GraphConvNetCell

from sklearn import decomposition


if torch.cuda.is_available():
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
else:
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor


class GraphConvNet3(nn.Module):

    def __init__(self, net_parameters):

        super(GraphConvNet3, self).__init__()

        # parameters
        D = net_parameters['D']
        n_components = net_parameters['n_components']
        H = net_parameters['H']
        L = net_parameters['L']

        # vector of hidden dimensions
        net_layers = []
        for layer in range(L):
            net_layers.append(H)

        # CL cells
        # NOTE: Each graph convnet cell uses *TWO* convolutional operations
        net_layers_extended = [D] + net_layers  # include embedding dim
        L = len(net_layers)
        list_of_gnn_cells = []  # list of NN cells
        for layer in range(L // 2):
            Hin, Hout = net_layers_extended[2 * layer], net_layers_extended[2 * layer + 2]
            list_of_gnn_cells.append(GraphConvNetCell(Hin, Hout))

        # register the cells for pytorch
        self.gnn_cells = nn.ModuleList(list_of_gnn_cells)

        # fc
        Hfinal = net_layers_extended[-1]
        self.fc = nn.Linear(Hfinal, n_components)

        # init
        self.init_weights_Graph_OurConvNet(Hfinal, n_components, 1)

        # print('\nnb of hidden layers=', L)
        # print('dim of layers (w/ embed dim)=', net_layers_extended)
        # print('\n')

        # class variables
        self.L = L
        self.D = D
        self.net_layers_extended = net_layers_extended

    def init_weights_Graph_OurConvNet(self, Fin_fc, Fout_fc, gain):

        scale = gain * np.sqrt(2.0 / Fin_fc)
        self.fc.weight.data.uniform_(-scale, scale)
        self.fc.bias.data.fill_(0)

    def forward(self, G):
        # Data matrix
        x = G.data

        # Unroll the image vector
        x = x.view(x.shape[0], -1)
        
        x = decomposition.TruncatedSVD(n_components=self.D).fit_transform(x)

        # Pass raw data matrix X directly as input
        x = Variable(torch.FloatTensor(x).type(dtypeFloat), requires_grad=False)

        # graph operators
        # Edge = start vertex to end vertex
        # E_start = E x V mapping matrix from edge index to corresponding start vertex
        # E_end = E x V mapping matrix from edge index to corresponding end vertex
        E_start = G.edge_to_starting_vertex
        E_end = G.edge_to_ending_vertex
        E_start = torch.from_numpy(E_start.toarray()).type(dtypeFloat)
        E_end = torch.from_numpy(E_end.toarray()).type(dtypeFloat)
        E_start = Variable(E_start, requires_grad=False)
        E_end = Variable(E_end, requires_grad=False)

        for layer in range(self.L // 2):
            gnn_layer = self.gnn_cells[layer]
            x = gnn_layer(x, E_start, E_end)  # V x Hfinal

        # FC
        x = self.fc(x)

        return x

    def loss(self, y, y_target):
        # L2 loss
        loss = nn.MSELoss()(y, y_target)

        return loss

    def pairwise_loss(self, y, y_target, W):
        distances_1 = y_target[W.row, :] - y_target[W.col, :]
        distances_2 = y[W.row, :] - y[W.col, :]
        loss = torch.mean(torch.pow(distances_1.norm(dim=1) - distances_2.norm(dim=1), 2))

        return loss

    def update(self, lr):

        update = torch.optim.Adam(self.parameters(), lr=lr)

        return update

    def update_learning_rate(self, optimizer, lr):

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        return optimizer

    def nb_param(self):
        return self.nb_param


In [13]:
# Graph net
net = GraphConvNet3(net_parameters)
root = 'results/mnist_tsne4_pca/'
filename = root + 'graph_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

similarity_tracker, trust_tracker, one_nn_tracker, time_tracker = evaluate_net(net, all_test_data)
print("Similarity loss = {}".format(np.average(similarity_tracker)))
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Similarity loss = 239.26064496621643
Trustworthy score = 0.7155905432916881
1-NN generalisation error = 0.6684939197182875
Average time to compute (s) = 0.36324966807973397


In [10]:
# Simple conv net
net = SimpleConvNet(net_parameters)
root = 'results/mnist_tsne2/'
filename = root + 'conv_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

similarity_tracker, trust_tracker, one_nn_tracker, time_tracker = evaluate_net(net, all_test_data)
print("Similarity loss = {}".format(np.average(similarity_tracker)))
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Similarity loss = 314.02360915456586
Trustworthy score = 0.5195835651987438
1-NN generalisation error = 0.8855259485903647
Average time to compute (s) = 0.6071047530096257
